In [ ]:
# This takes a dataset, prepares the data, splits it, tries out different ML models,
# picks the best one based on test data, checks it on validation data, saves the pipeline,
# and maybe tries out a web app

# (In the style of Chpt. 2 exercises, p. 84)

In [59]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from pandas.plotting import scatter_matrix
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

%matplotlib qt

### Function defs

In [15]:
def load_data(data_path=DATA_PATH):
    csv_path = os.path.join(DATA_PATH, "heart.csv")
    return pd.read_csv(csv_path)

In [37]:
def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

### Read in data

In [14]:
STEM = "/Users/bandari/Documents/git.repos/ml/"
DATA_PATH = os.path.join(STEM, "datasets")

### View it 

In [19]:
df = load_data()

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


In [25]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


### Some pre-processing before splitting

In [28]:
# turn some string quantities into numerical ones ('encode')
# and show what the numbers stand for

# remove NaNs first
#df["Expedition"].fillna("unk", inplace=True)

ordinal_encoder = OrdinalEncoder()
df["Sex_encoded"] = ordinal_encoder.fit_transform(df[["Sex"]])
print(ordinal_encoder.categories_)
print("----")
df["ChestPainType_encoded"] = ordinal_encoder.fit_transform(df[["ChestPainType"]])
print(ordinal_encoder.categories_)
print("----")
df["RestingECG_encoded"] = ordinal_encoder.fit_transform(df[["RestingECG"]])
print(ordinal_encoder.categories_)
print("----")
df["ExerciseAngina_encoded"] = ordinal_encoder.fit_transform(df[["ExerciseAngina"]])
print(ordinal_encoder.categories_)
print("----")
df["ST_Slope_encoded"] = ordinal_encoder.fit_transform(df[["ST_Slope"]])
print(ordinal_encoder.categories_)
print("----")

[array(['F', 'M'], dtype=object)]
----
[array(['ASY', 'ATA', 'NAP', 'TA'], dtype=object)]
----
[array(['LVH', 'Normal', 'ST'], dtype=object)]
----
[array(['N', 'Y'], dtype=object)]
----
[array(['Down', 'Flat', 'Up'], dtype=object)]
----


In [35]:
# scatter matrix

scatter_matrix(df)

array([[<AxesSubplot:xlabel='Age', ylabel='Age'>,
        <AxesSubplot:xlabel='RestingBP', ylabel='Age'>,
        <AxesSubplot:xlabel='Cholesterol', ylabel='Age'>,
        <AxesSubplot:xlabel='FastingBS', ylabel='Age'>,
        <AxesSubplot:xlabel='MaxHR', ylabel='Age'>,
        <AxesSubplot:xlabel='Oldpeak', ylabel='Age'>,
        <AxesSubplot:xlabel='HeartDisease', ylabel='Age'>,
        <AxesSubplot:xlabel='Sex_encoded', ylabel='Age'>,
        <AxesSubplot:xlabel='ChestPainType_encoded', ylabel='Age'>,
        <AxesSubplot:xlabel='RestingECG_encoded', ylabel='Age'>,
        <AxesSubplot:xlabel='ExerciseAngina_encoded', ylabel='Age'>,
        <AxesSubplot:xlabel='ST_Slope_encoded', ylabel='Age'>],
       [<AxesSubplot:xlabel='Age', ylabel='RestingBP'>,
        <AxesSubplot:xlabel='RestingBP', ylabel='RestingBP'>,
        <AxesSubplot:xlabel='Cholesterol', ylabel='RestingBP'>,
        <AxesSubplot:xlabel='FastingBS', ylabel='RestingBP'>,
        <AxesSubplot:xlabel='MaxHR', ylabel='Re

### Split into training and validation datasets

In [40]:
# note this is before feature scaling

train_set, test_set = split_train_test(df, 0.2)

In [47]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 735 entries, 916 to 154
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Age                     735 non-null    int64  
 1   Sex                     735 non-null    object 
 2   ChestPainType           735 non-null    object 
 3   RestingBP               735 non-null    int64  
 4   Cholesterol             735 non-null    int64  
 5   FastingBS               735 non-null    int64  
 6   RestingECG              735 non-null    object 
 7   MaxHR                   735 non-null    int64  
 8   ExerciseAngina          735 non-null    object 
 9   Oldpeak                 735 non-null    float64
 10  ST_Slope                735 non-null    object 
 11  HeartDisease            735 non-null    int64  
 12  Sex_encoded             735 non-null    float64
 13  ChestPainType_encoded   735 non-null    float64
 14  RestingECG_encoded      735 non-null    

### Feature-scale only the training set

In [50]:
# make a 'pipeline' to feature scale

num_pipeline = Pipeline([('std_scaler', StandardScaler())])

# drop non-numerical features
train_set_dropped = train_set.drop(columns=["Sex","ChestPainType","RestingECG","ExerciseAngina","ST_Slope"], inplace=False)

train_set_preprocessed = num_pipeline.fit_transform(train_set_dropped)

# the syntax here is necessary to restore column keys
train_set_preprocessed = pd.DataFrame(num_pipeline.fit_transform(train_set_dropped),columns = train_set_dropped.columns)

### Separate training set into predictors and labels

In [57]:
# predictors: the things that help us predict
# labels: the things we want to predict

data_predictors = train_set_preprocessed.drop("HeartDisease", axis=1)
data_labels = train_set_preprocessed["HeartDisease"].copy()

### Try individual ML models with a grid search for each

In [69]:
# linear model

lin_reg = LinearRegression()
lin_reg.fit(data_predictors, data_labels)
data_preds = lin_reg.predict(data_predictors)

In [70]:
type(data_preds)

numpy.ndarray

In [66]:
# see a few examples

print("Predictions: ", lin_reg.predict(data_predictors)[:5])
print("Labels: ", list(data_labels)[:5])

Predictions:  [-0.52958598  0.52690215  1.40119342  1.70258794  0.80287252]
Labels:  [0.887874522600582, 0.887874522600582, 0.887874522600582, 0.887874522600582, 0.887874522600582]


In [77]:
# How good are the predictions?

plt.hist(data_preds[np.where(data_labels > 0)], color="k", alpha=0.5)
plt.axvline(x=0.8878745226, color="k", alpha=1)
plt.hist(data_preds[np.where(data_labels < 0)], color="blue", alpha=0.5)
plt.axvline(x=-1.126285, color="blue", alpha=1)
plt.legend()
plt.show()

No handles with labels found to put in legend.


In [78]:
# see coefficients 
# (ref. https://scikit-learn.org/stable/modules/linear_model.html )

lin_reg.coef_

array([ 0.06804422,  0.01830615, -0.12301281,  0.12441496, -0.05732069,
        0.10024127,  0.15357378, -0.19412729, -0.02351416,  0.17686366,
       -0.29077322])

### Which model appears to perform best?

### Check performance of all models on validation data. 
### Is the best model based on the training data also the best based on the validation data?

### Agglomerate into a single pipeline, and save the best one

### Try a web app (may need to read bookmarked websites)